# Explorons l'API [Carbon Interface](https://www.carboninterface.com)

## Obtenir une clé pour l'API

Allez sur [https://www.carboninterface.com/users/sign_up](https://www.carboninterface.com/users/sign_up) et inscrivez-vous pour obtenir votre clé.

Créez un fichier nommé `.env` (le point au début du nom de fichier est important) et dedans écrivez :

> APIKEY=<votre clé api>

## Installer les modules python nécessaires

In [1]:
import os

In [ ]:
os.system("pip install requests python-dotenv")

## Imports

In [2]:
import requests
from dotenv import load_dotenv

## Charger la clé API grâce à python-dotenv

In [3]:
load_dotenv()
# cette ligne lit votre clé depuis le fichier.env
key = os.getenv('APIKEY')

## Test de la clé avec `requests.get()`

In [4]:
url = "https://www.carboninterface.com/api/v1/"
auth_endpoint = "auth"

headers = {'Authorization': f"Bearer {key}", "Content-Type": "application/json"}

La cellule ci-dessous doit sortir `{'message': 'auth successful'}`

In [5]:
response = requests.get(url+auth_endpoint, headers=headers)
response.json()

{'message': 'auth successful'}

## Test de la partie `estimates` de l'API avec `requests.post()`

Pour trouver les codes à trois lettres des différents aéroports, vous pouvez chercher sur cette page [https://www.iata.org/en/publications/directories/code-search/](https://www.iata.org/en/publications/directories/code-search/), dans la section **Search Location Codes**.

In [6]:
endpoint = 'estimates'
json = {
        "type": "flight",
        "passengers": 2,
        "legs": [
          {"departure_airport": "lys", "destination_airport": "atl"},
          {"departure_airport": "atl", "destination_airport": "lys"}
        ]
      }

In [7]:
response = requests.post(url+endpoint, json=json, headers=headers)
response.json()

{'data': {'id': '74e2a273-bb20-497a-ab9f-a59bda525c3c',
  'type': 'estimate',
  'attributes': {'passengers': 2,
   'legs': [{'departure_airport': 'LYS', 'destination_airport': 'ATL'},
    {'departure_airport': 'ATL', 'destination_airport': 'LYS'}],
   'distance_value': 14990.81,
   'distance_unit': 'km',
   'estimated_at': '2022-05-09T12:04:04.221Z',
   'carbon_g': 4405779,
   'carbon_lb': 9713.08,
   'carbon_kg': 4405.78,
   'carbon_mt': 4.41}}}

In [8]:
data = response.json()['data']

In [9]:
data['attributes']['carbon_kg']

4405.78

## Test de la partie `vehicle_makes`

### Trouver la marque de mon véhicule

In [10]:
vehicle_makes_endpoint = 'vehicle_makes/'

makes = requests.get(url+vehicle_makes_endpoint, headers=headers)

In [11]:
makes.json()[:3]

[{'data': {'id': '5f266411-5bb1-4b91-b044-9707426df630',
   'type': 'vehicle_make',
   'attributes': {'name': 'Alfa Romeo', 'number_of_models': 69}}},
 {'data': {'id': '4c1e16e1-7967-4394-b3cb-15f4577dffa1',
   'type': 'vehicle_make',
   'attributes': {'name': 'Ferrari', 'number_of_models': 243}}},
 {'data': {'id': 'c0d79b67-76e8-442d-b105-2c73501948a9',
   'type': 'vehicle_make',
   'attributes': {'name': 'Dodge', 'number_of_models': 2608}}}]

Les données de la réponse sont une longue liste de dictionnaires, contenant chacun un dictionnaire :)

### Définissons une fonction qui va chercher l'`id` d'un constructeur automobile à partir de son nom.

In [15]:
def find_make_or_car_id(name, response):
    if not response.status_code == 200:
        return None
    for elem in response.json():
        if elem['data']['attributes']['name'] == name:
            return elem['data']['id']
    


In [13]:
make_name = 'Renault'

In [14]:
find_make_or_car_id(make_name, makes)

'6e8ef1b4-3714-4024-85f5-c7f5a35b0e0c'

### Trouver mon véhicule à partir du constructeur

In [16]:
vehicles_endpoint = find_make_or_car_id(make_name, makes) + "/vehicle_models"

In [17]:
cars = requests.get(url+vehicle_makes_endpoint+vehicles_endpoint, headers=headers)

In [18]:
cars.json()[:3]

[{'data': {'id': 'ff4b4c0f-d26f-4ed2-9b14-f1fb4e44e629',
   'type': 'vehicle_model',
   'attributes': {'name': 'Alliance Convertible',
    'year': 1986,
    'vehicle_make': 'Renault'}}},
 {'data': {'id': 'a2603423-c451-48e2-a8f7-929efa5a1d54',
   'type': 'vehicle_model',
   'attributes': {'name': 'Alliance Convertible',
    'year': 1986,
    'vehicle_make': 'Renault'}}},
 {'data': {'id': '58d0f6c2-40c8-4553-8ed7-dcdaa93f7cfb',
   'type': 'vehicle_model',
   'attributes': {'name': 'Fuego', 'year': 1986, 'vehicle_make': 'Renault'}}}]

**NB** : On réutilise la fonction qui cherche une marque de constructeur dans la réponse, la structure des listes de dictionnaires est identique.

In [19]:
find_make_or_car_id('Fuego', cars) 

'58d0f6c2-40c8-4553-8ed7-dcdaa93f7cfb'

### Définissions une fonction qui fait la requête des véhicules d'une marque à partir de son nom

In [21]:
def get_car_maker_response(make_name):
    vehicle_makes_endpoint = 'vehicle_makes/'
    makes = requests.get(url+vehicle_makes_endpoint, headers=headers)
    maker_id = find_make_or_car_id(make_name, makes)
    if not maker_id:
        return None
   
    vehicles_endpoint = maker_id + "/vehicle_models"
    cars = requests.get(url+vehicle_makes_endpoint+vehicles_endpoint, headers=headers)
    return cars
  
        
    

In [23]:
exemple = get_car_maker_response('Dacia')

In [24]:
exemple.json()

[{'data': {'id': '477b5e49-6886-4925-aabb-36f1c433e09d',
   'type': 'vehicle_model',
   'attributes': {'name': 'Coupe', 'year': 1988, 'vehicle_make': 'Dacia'}}},
 {'data': {'id': '5b214d6e-df9d-497a-a44b-1b82852cad15',
   'type': 'vehicle_model',
   'attributes': {'name': 'Sedan', 'year': 1988, 'vehicle_make': 'Dacia'}}},
 {'data': {'id': '058c049e-c211-43b3-90e0-bf518c7f605a',
   'type': 'vehicle_model',
   'attributes': {'name': 'Station Wagon',
    'year': 1988,
    'vehicle_make': 'Dacia'}}}]

## Lister les noms de modèles pour une marque

In [25]:
def get_car_names(make_name):
    car_maker_response = get_car_maker_response(make_name)
    car_names = [car['data']['attributes']['name'] for car in car_maker_response.json()]
    return list(set(car_names))

In [27]:
get_car_names('Mitsubishi')

['Truck 2WD',
 'Endeavor',
 'Diamante',
 'Diamante Wagon',
 'Precis',
 'Space Wagon',
 'Endeavor AWD',
 'Lancer Sportback',
 'Starion',
 'Endeavor 4WD',
 'Raider Pickup 4WD',
 'Mirage Wagon',
 'Lancer AWD',
 'Wagon',
 'Nativa 2WD (Puerto Rico Only)',
 'Expo.LRV',
 'Nativa 2WD(Puerto Rico Only)',
 'Endeavor 2WD',
 'Montero Sport 4WD',
 'Outlander Sport 2WD',
 'Nativa 2WD',
 'Outlander PHEV',
 'Eclipse Spyder',
 'Eclipse Cross 4WD',
 'Sigma',
 'Eclipse',
 'Montero',
 'Eclipse Cross ES 2WD',
 'Mirage',
 '3000 GT Spyder',
 'Galant',
 'Montero 4WD',
 'Truck 4WD',
 'Outlander Sport 4WD',
 'Expo',
 'Montero Sport 2WD',
 'Outlander 4WD',
 'Mirage G4',
 'Tredia',
 'Nativa 4WD (Puerto Rico Only)',
 'Galant Sigma',
 'Cordia',
 'Diamante Sedan',
 'Outlander 2WD',
 'Eclipse Cross 2WD',
 'Space Wagon 4WD',
 'Lancer Evolution',
 'i-MiEV',
 'Lancer 4WD',
 'Lancer',
 'Raider Pickup 2WD',
 'Eclipse Cross ES 4WD',
 'Van',
 'Nativa 4WD',
 '3000 GT',
 'Eclipse Convertible']

## Requête pour trouver l'estimation des emissions d'un trajet en voiture

In [28]:
def estimate_trip(make_name, car_name, distance):
    estimates_endpoint = 'estimates'
    car_maker_response = get_car_maker_response(make_name)
        
    if not car_maker_response:
        return None
    
    request_body = {
            "type": "vehicle",
            "distance_unit": "km",
            "distance_value": distance,
            "vehicle_model_id": find_make_or_car_id(car_name, car_maker_response)
          }

    response = requests.post(url+estimates_endpoint, json=request_body, headers=headers)
    emission = response.json()['data']['attributes']['carbon_kg']
    result = f"Carbon emissions for a {distance} km trip driving a {make_name} {car_name}: {emission} kg of CO2."
    print(result)

In [29]:
estimate_trip("Renault", "Fuego", 100)

Carbon emissions for a 100 km trip driving a Renault Fuego: 29.06 kg of CO2.


In [30]:
estimate_trip("Tesla", "Model S AWD - P100D", 100)

Carbon emissions for a 100 km trip driving a Tesla Model S AWD - P100D: 0.0 kg of CO2.


In [31]:
estimate_trip('Dacia', 'Sedan', 300)

Carbon emissions for a 300 km trip driving a Dacia Sedan: 72.03 kg of CO2.


<div class='alert alert-success'>
    
Si vous voulez échanger sur cette API et comment l'intégrer à votre enseignement ou recherche, n’hésitez pas à me solliciter philippe.lemaire@insa-lyon.fr

</div>